# 영상처리와 딥러닝 11주차 autoencoder 실습 참고자료

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=green)

참고할만한 원본 소스

**Filled notebook:**
[![View on Github](https://img.shields.io/static/v1.svg?logo=github&label=Repo&message=View%20On%20Github&color=lightgrey)](https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial9/AE_CIFAR10.ipynb)
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial9/AE_CIFAR10.ipynb)   
**Pre-trained models:**
[![View files on Github](https://img.shields.io/static/v1.svg?logo=github&label=Repo&message=View%20On%20Github&color=lightgrey)](https://github.com/phlippe/saved_models/tree/main/tutorial9)
[![GoogleDrive](https://img.shields.io/static/v1.svg?logo=google-drive&logoColor=yellow&label=GDrive&message=Download&color=yellow)](https://drive.google.com/drive/folders/1acxmt1AK9iUvCLMNrd7qcI0WjJbQYxyv?usp=sharing)     
**Recordings:**
[![YouTube - Part 1](https://img.shields.io/static/v1.svg?logo=youtube&label=YouTube&message=Part%201&color=red)](https://youtu.be/E2d8NRYt2e4)
[![YouTube - Part 2](https://img.shields.io/static/v1.svg?logo=youtube&label=YouTube&message=Part%202&color=red)](https://youtu.be/3UrX2mTY610)

이 튜토리얼에서는 오토인코더(AE)에 대해 자세히 살펴보겠습니다. 오토인코더는 이미지와 같은 입력 데이터를 더 작은 특징 벡터로 인코딩한 후 디코더라고 하는 두 번째 신경망에 의해 재구성하도록 훈련됩니다. 특징 벡터는 입력 데이터를 더 적은 양의 특징으로 압축하는 것을 목표로 하기 때문에 네트워크의 "병목"이라고 합니다. 이 속성은 특히 데이터를 압축하거나 픽셀 수준 비교를 넘어 메트릭에서 이미지를 비교할 때 많은 응용 프로그램에서 유용합니다. 오토인코더 프레임워크에 대해 배우는 것 외에도 높이와 너비의 피쳐 맵을 확장하기 위해 작동하는 "디콘볼루션"(또는 전치된 컨볼루션) 연산자도 볼 수 있습니다. 이러한 디콘볼루션 네트워크는 우리가 작은 특징 벡터에서 시작하고 전체 크기의 이미지를 출력해야 하는 모든 곳에서 필요합니다(예: VAE, GAN 또는 초해상도 응용 프로그램).

훈련 코드 오버헤드를 줄이기 위해 PyTorch Lightning을 사용할 것입니다.

In [1]:
## Standard libraries
import os
import json
import math
import numpy as np

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Tensorboard extension (for visualization purposes later)
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = "../data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "../saved_models/tutorial9"

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

<ipython-input-1-9c0493687520>:11: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export
INFO:lightning_fabric.utilities.seed:Seed set to 42


Device: cuda:0


다운로드해야 하는 4개의 사전 훈련된 모델이 있습니다. 필요한 경우 변수 `DATASET_PATH` 및 `CHECKPOINT_PATH`를 조정해야 합니다.

In [2]:
import urllib.request
from urllib.error import HTTPError
# Github URL where saved models are stored for this tutorial
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial9/"
# Files to download
pretrained_files = ["cifar10_64.ckpt", "cifar10_128.ckpt", "cifar10_256.ckpt", "cifar10_384.ckpt"]
# Create checkpoint path if it doesn't exist yet
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# For each file, check whether it already exists. If not, try downloading it.
for file_name in pretrained_files:
    file_path = os.path.join(CHECKPOINT_PATH, file_name)
    if not os.path.isfile(file_path):
        file_url = base_url + file_name
        print(f"Downloading {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
        except HTTPError as e:
            print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)

이 튜토리얼에서는 CIFAR10 데이터로 작업합니다. CIFAR10에서 각 이미지는 3개의 색상 채널을 가지며 32x32픽셀 크기입니다. 오토인코더는 이미지를 확률적으로 모델링하는 제약이 없기 때문에 VAE보다 훨씬 더 복잡한 이미지 데이터(예: 흑백 대신 3개의 색상 채널)를 작업할 수 있습니다.
이미 다른 디렉토리에 CIFAR10을 다운로드한 경우, 다른 다운로드를 방지하기 위해 DATASET_PATH를 적절하게 설정해야 합니다.

[Tutorial 5](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial5/Inception_ResNet_DenseNet.html)(CNN 분류)와 같은 CIFAR10에 대한 이전 자습서와 달리, 우리는 다음을 사용하여 데이터를 명시적으로 정규화하지 않습니다. 평균은 0이고 표준은 1이지만 대략적으로 -1과 1 사이의 데이터를 스케일링하여 추정합니다. 이는 범위를 제한하면 이미지 예측/재구성 작업이 더 쉬워지기 때문입니다.

In [3]:
# Transformations applied on each image => only make them a tensor
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,))])

# Loading the training dataset. We need to split it into a training and validation part
train_dataset = CIFAR10(root=DATASET_PATH, train=True, transform=transform, download=True)
pl.seed_everything(42)
train_set, val_set = torch.utils.data.random_split(train_dataset, [45000, 5000])

# Loading the test set
test_set = CIFAR10(root=DATASET_PATH, train=False, transform=transform, download=True)

# We define a set of data loaders that we can use for various purposes later.
train_loader = data.DataLoader(train_set, batch_size=256, shuffle=True, drop_last=True, pin_memory=True, num_workers=4)
val_loader = data.DataLoader(val_set, batch_size=256, shuffle=False, drop_last=False, num_workers=4)
test_loader = data.DataLoader(test_set, batch_size=256, shuffle=False, drop_last=False, num_workers=4)

def get_train_images(num):
    return torch.stack([train_dataset[i][0] for i in range(num)], dim=0)

100%|██████████| 170498071/170498071 [00:12<00:00, 13378083.19it/s]


Extracting ../data/cifar-10-python.tar.gz to ../data


INFO:lightning_fabric.utilities.seed:Seed set to 42


Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## Building the autoencoder

일반적으로 자동 인코더는 입력 $x$를 저차원 특징 벡터 $z$에 매핑하는 **인코더**와 입력 $\hat{x}$를 재구성하는 **디코더**로 구성됩니다. $z$. $x$와 $\hat{x}$를 비교하고 매개변수를 최적화하여 $x$와 $\hat{x}$ 사이의 유사성을 높이는 방식으로 모델을 훈련합니다. autoencoder 프레임워크의 작은 그림은 아래를 참조하십시오.

<center width="100%"><img src="https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial9/autoencoder_visualization.svg?raw=1" style="display: block; margin-left: auto; margin-right: auto;" width="650px"/></center>

먼저 인코더를 구현하는 것으로 시작합니다. 인코더는 strided convolution을 사용하여 이미지를 레이어별로 축소하는 deep convolutional network로 효과적으로 구성됩니다. 이미지를 세 번 축소한 후 특징을 평평하게 하고 선형 레이어를 적용합니다. 따라서 잠재 표현 $z$는 유연하게 선택할 수 있는 *d* 크기의 벡터입니다.

In [ ]:
class Encoder(nn.Module):

    def __init__(self,
                 num_input_channels : int,
                 base_channel_size : int,
                 latent_dim : int,
                 act_fn : object = nn.GELU):
        """
        Inputs:
            - num_input_channels : Number of input channels of the image. For CIFAR, this parameter is 3
            - base_channel_size : Number of channels we use in the first convolutional layers. Deeper layers might use a duplicate of it.
            - latent_dim : Dimensionality of latent representation z
            - act_fn : Activation function used throughout the encoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.net = nn.Sequential(
            nn.Conv2d(num_input_channels, c_hid, kernel_size=3, padding=1, stride=2), # 32x32 => 16x16
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(c_hid, 2*c_hid, kernel_size=3, padding=1, stride=2), # 16x16 => 8x8
            act_fn(),
            nn.Conv2d(2*c_hid, 2*c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(2*c_hid, 2*c_hid, kernel_size=3, padding=1, stride=2), # 8x8 => 4x4
            act_fn(),
            nn.Flatten(), # Image grid to single feature vector
            nn.Linear(2*16*c_hid, latent_dim)
        )

    def forward(self, x):
        return self.net(x)

여기서는 Batch Normalization을 적용하지 않습니다. 이는 각 이미지의 인코딩이 다른 모든 이미지와 독립적이기를 원하기 때문입니다. 그렇지 않으면 원하지 않는 인코딩 또는 디코딩에 상관 관계가 발생할 수 있습니다. 일부 구현에서는 Batch Normalization이 정규화의 한 형태로 사용될 수도 있기 때문에 여전히 사용 중인 것을 볼 수 있습니다. 그럼에도 불구하고 필요한 경우 인스턴스 정규화 또는 계층 정규화와 같은 다른 정규화 기술을 사용하는 것이 좋습니다. 모델의 작은 크기를 감안할 때 지금은 정규화를 무시할 수 있습니다.

디코더는 인코더의 뒤집힌 버전입니다. 유일한 차이점은 기능을 확장하기 위해 strided convolution을 transposed convolution(즉, deconvolution)으로 대체한다는 것입니다. 커널 크기가 3, stride 2, 패딩이 1인 `nn.ConvTranspose2d` 레이어의 그림은 아래를 참조하세요(그림 출처 - [Vincent Dumoulin and Francesco Visin](https://arxiv.org/abs/1603.07285)). :

<center width="100%"><img src="https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial9/deconvolution.gif?raw=1" width="250px"></center>

$3\times3$ 크기의 입력에 대해 $5\times5$의 출력을 얻습니다. 그러나 실제로 컨볼루션의 역 연산을 수행하려면 레이어가 2의 인수로 입력 모양을 확장하도록 해야 합니다(예: $4\times4\to8\times8$). 이를 위해 출력 모양에 추가 값을 추가하는 'output_padding' 매개변수를 지정할 수 있습니다. 이것으로 제로 패딩을 수행하지 않고 계산을 위해 출력 모양을 늘립니다.

전반적으로 디코더는 다음과 같이 구현할 수 있습니다.

In [ ]:
class Decoder(nn.Module):

    def __init__(self,
                 num_input_channels : int,
                 base_channel_size : int,
                 latent_dim : int,
                 act_fn : object = nn.GELU):
        """
        Inputs:
            - num_input_channels : Number of channels of the image to reconstruct. For CIFAR, this parameter is 3
            - base_channel_size : Number of channels we use in the last convolutional layers. Early layers might use a duplicate of it.
            - latent_dim : Dimensionality of latent representation z
            - act_fn : Activation function used throughout the decoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.linear = nn.Sequential(
            nn.Linear(latent_dim, 2*16*c_hid),
            act_fn()
        )
        self.net = nn.Sequential(
            nn.ConvTranspose2d(2*c_hid, 2*c_hid, kernel_size=3, output_padding=1, padding=1, stride=2), # 4x4 => 8x8
            act_fn(),
            nn.Conv2d(2*c_hid, 2*c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(2*c_hid, c_hid, kernel_size=3, output_padding=1, padding=1, stride=2), # 8x8 => 16x16
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(c_hid, num_input_channels, kernel_size=3, output_padding=1, padding=1, stride=2), # 16x16 => 32x32
            nn.Tanh() # The input images is scaled between -1 and 1, hence the output has to be bounded as well
        )

    def forward(self, x):
        x = self.linear(x)
        x = x.reshape(x.shape[0], -1, 4, 4)
        x = self.net(x)
        return x

여기에서 선택한 인코더 및 디코더 네트워크는 비교적 간단합니다. 일반적으로 특히 ResNet 기반 아키텍처를 사용할 때 더 복잡한 네트워크가 적용됩니다. 예를 들어, [VQ-VAE](https://arxiv.org/abs/1711.00937)를 참조하십시오.

마지막 단계에서 인코더와 디코더를 함께 오토인코더 아키텍처에 추가합니다. 필요한 교육 코드를 단순화하기 위해 오토인코더를 PyTorch Lightning Module로 구현합니다.

In [ ]:
class Autoencoder(pl.LightningModule):

    def __init__(self,
                 base_channel_size: int,
                 latent_dim: int,
                 encoder_class : object = Encoder,
                 decoder_class : object = Decoder,
                 num_input_channels: int = 3,
                 width: int = 32,
                 height: int = 32):
        super().__init__()
        # Saving hyperparameters of autoencoder
        self.save_hyperparameters()
        # Creating encoder and decoder
        self.encoder = encoder_class(num_input_channels, base_channel_size, latent_dim)
        self.decoder = decoder_class(num_input_channels, base_channel_size, latent_dim)
        # Example input array needed for visualizing the graph of the network
        self.example_input_array = torch.zeros(2, num_input_channels, width, height)

    def forward(self, x):
        """
        The forward function takes in an image and returns the reconstructed image
        """
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

    def _get_reconstruction_loss(self, batch):
        """
        Given a batch of images, this function returns the reconstruction loss (MSE in our case)
        """
        x, _ = batch # We do not need the labels
        x_hat = self.forward(x)
        loss = F.mse_loss(x, x_hat, reduction="none")
        loss = loss.sum(dim=[1,2,3]).mean(dim=[0])
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        # Using a scheduler is optional but can be helpful.
        # The scheduler reduces the LR if the validation performance hasn't improved for the last N epochs
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                         mode='min',
                                                         factor=0.2,
                                                         patience=20,
                                                         min_lr=5e-5)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

    def training_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log('val_loss', loss)

    def test_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log('test_loss', loss)

손실 함수의 경우 평균 제곱 오차(MSE)를 사용합니다. 평균 제곱 오차는 네트워크가 추정치가 멀리 떨어져 있는 픽셀 값에 특별한 주의를 기울이도록 합니다. 재구성할 때 128 대신 127을 예측하는 것은 중요하지 않지만 0을 128과 혼동하는 것은 훨씬 더 나쁩니다. VAE와 달리 픽셀 값당 확률을 예측하지 않고 대신 distance 측정을 사용합니다. 이것은 많은 매개변수를 저장하고 훈련을 단순화합니다. 픽셀당 더 나은 직관을 얻기 위해 배치 차원에 대해 평균 제곱 오차 합계를 보고합니다(다른 모든 평균/합계는 동일한 결과/매개변수로 이어짐).


이미지 재구성에 대한 평가척도(점수)로써 널리쓰이는것은 PSNR입니다. PSNR은 이미지 최대값의 제곱을  MSE로 나눈 후 log 스케일로 표현하는 메트릭입니다. 일반적으로 40dB이상은 좋은 품질, 25dB 이하는 나쁜 품질의 이미지를 뜻합니다.

그러나 MSE/PSNR에는 몇 가지 상당한 단점도 있습니다. 일반적으로 MSE는 매우 낮은 오류를 유발하므로 작은 노이즈/고주파 패턴이 제거되는 흐릿한 이미지로 이어집니다. 실제 이미지가 재구성되도록 보장하기 위해 여러 작업에서 수행된 것처럼 Generative Adversarial Networks와 오토인코더를 결합할 수 있습니다(예: [여기](https://arxiv.org/abs/1704.02304), [여기](https 참조). //arxiv.org/abs/1511.05644) 또는 이 [슬라이드](http://elarosca.net/slides/iccv_autoencoder_gans.pdf)).

또한 MSE를 사용하여 두 이미지를 비교하는 것이 반드시 시각적 유사성을 반영하는 것은 아닙니다. 예를 들어, 자동 인코더가 오른쪽과 아래쪽으로 1픽셀 이동한 이미지를 재구성한다고 가정합니다. 이미지는 거의 동일하지만 이미지의 절반에 대해 일정한 픽셀 값을 예측하는 것보다 더 높은 손실을 얻을 수 있습니다(아래 코드 참조). 이 문제에 대한 예제 솔루션에는 별도의 사전 훈련된 CNN을 사용하고 원래 픽셀 수준 비교 대신 하위 레이어의 시각적 특징 거리를 거리 측정으로 사용하는 것이 포함됩니다.

  

In [ ]:
def compare_imgs(img1, img2, title_prefix=""):
    # Calculate MSE loss between both images
    mse = F.mse_loss(img1, img2, reduction="sum")
    loss = 10*torch.log10(255*255/F.mse_loss(img1, img2, reduction="sum"))
    # Plot images for visual comparison
    grid = torchvision.utils.make_grid(torch.stack([img1, img2], dim=0), nrow=2, normalize=True, range=(-1,1))
    grid = grid.permute(1, 2, 0)
    plt.figure(figsize=(4,2))
    plt.title(f"{title_prefix} MSE: {mse.item():4.2f},      PSNR: {loss.item():4.2f}")
    plt.imshow(grid)
    plt.axis('off')
    plt.show()

for i in range(2):
    # Load example image
    img, _ = train_dataset[i]
    img_mean = img.mean(dim=[1,2], keepdims=True)

    # Shift image by one pixel
    SHIFT = 1
    img_shifted = torch.roll(img, shifts=SHIFT, dims=1)
    img_shifted = torch.roll(img_shifted, shifts=SHIFT, dims=2)
    img_shifted[:,:1,:] = img_mean
    img_shifted[:,:,:1] = img_mean
    compare_imgs(img, img_shifted, "Shifted -")

    # Set half of the image to zero
    img_masked = img.clone()
    img_masked[:,:img_masked.shape[1]//2,:] = img_mean
    compare_imgs(img, img_masked, "Masked -")

### Training the model

훈련하는 동안 우리는 모델이 만든 재구성을 보고 학습 진행 상황을 추적하려고 합니다. 이를 위해 PyTorch Lightning에서 $N$ epoch마다 텐서보드(tensorboard)에 재구성을 추가하는 콜백 객체를 구현합니다.

In [ ]:
class GenerateCallback(pl.Callback):

    def __init__(self, input_imgs, every_n_epochs=1):
        super().__init__()
        self.input_imgs = input_imgs # Images to reconstruct during training
        self.every_n_epochs = every_n_epochs # Only save those images every N epochs (otherwise tensorboard gets quite large)

    def on_epoch_end(self, trainer, pl_module):
        if trainer.current_epoch % self.every_n_epochs == 0:
            # Reconstruct images
            input_imgs = self.input_imgs.to(pl_module.device)
            with torch.no_grad():
                pl_module.eval()
                reconst_imgs = pl_module(input_imgs)
                pl_module.train()

            # Plot and add to tensorboard
            imgs = torch.stack([input_imgs, reconst_imgs], dim=1).flatten(0,1)
            grid = torchvision.utils.make_grid(imgs, nrow=2, normalize=True, range=(-1,1))
            trainer.logger.experiment.add_image("Reconstructions", grid, global_step=trainer.global_step)

이제 다양한 잠재 차원(latent dimensionality)을 가진 오토인코더를 훈련하고 테스트 점수와 유효성 검사 점수를 모두 반환하는 훈련 함수를 작성할 것입니다. 우리는 사전 훈련된 모델을 제공하며 특히 GPU가 없는 컴퓨터에서 작업할 때 이러한 모델을 사용할 것을 권장합니다.

In [ ]:
def train_cifar(latent_dim):
    # Create a PyTorch Lightning trainer with the generation callback
    trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, f"cifar10_{latent_dim}"),
                         gpus=1 if str(device).startswith("cuda") else 0,
                         max_epochs=500,
                         callbacks=[ModelCheckpoint(save_weights_only=True),
                                    GenerateCallback(get_train_images(8), every_n_epochs=10),
                                    LearningRateMonitor("epoch")])
    trainer.logger._log_graph = True         # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"cifar10_{latent_dim}.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = Autoencoder.load_from_checkpoint(pretrained_filename)
    else:
        model = Autoencoder(base_channel_size=32, latent_dim=latent_dim)
        trainer.fit(model, train_loader, val_loader)
    # Test best model on validation and test set
    val_result = trainer.test(model, test_dataloaders=val_loader, verbose=False)
    test_result = trainer.test(model, test_dataloaders=test_loader, verbose=False)
    result = {"test": test_result, "val": val_result}
    return model, result

### Comparing latent dimensionality

오토인코더를 훈련할 때 잠재 표현 $z$에 대한 차원을 선택해야 합니다. 잠재 차원이 높을수록 재구성이 더 잘 될 것으로 기대합니다. 그러나 자동 인코더의 개념은 데이터를 *압축*하는 것입니다. 따라서 우리는 차원을 낮게 유지하는 데에도 관심이 있습니다. 최상의 절충안을 찾기 위해 서로 다른 잠재 차원을 가진 여러 모델을 훈련할 수 있습니다. 원래 입력에는 $32\times 32\times 3 = 3072$ 픽셀이 있습니다. 이를 염두에 두고 잠재 차원에 대한 합리적인 선택은 64와 384 사이일 수 있습니다.

In [ ]:
model_dict = {}
for latent_dim in [64, 128, 256, 384]:
    model_ld, result_ld = train_cifar(latent_dim)
    model_dict[latent_dim] = {"model": model_ld, "result": result_ld}

모델을 훈련시킨 후 잠재 차원에 대한 재구성 손실을 플롯하여 이 두 속성이 어떻게 상관되는지 직관할 수 있습니다.

In [ ]:
latent_dims = sorted([k for k in model_dict])
val_scores = [model_dict[k]["result"]["val"][0]["test_loss"] for k in latent_dims]

fig = plt.figure(figsize=(6,4))
plt.plot(latent_dims, val_scores, '--', color="#000", marker="*", markeredgecolor="#000", markerfacecolor="y", markersize=16)
plt.xscale("log")
plt.xticks(latent_dims, labels=latent_dims)
plt.title("Reconstruction error over latent dimensionality", fontsize=14)
plt.xlabel("Latent dimensionality")
plt.ylabel("Reconstruction error")
plt.minorticks_off()
plt.ylim(0,100)
plt.show()

우리가 처음에 예상했듯이, 재건 손실은 잠재 차원이 증가함에 따라 줄어듭니다. 우리의 모델과 설정에서 두 속성은 기하급수적으로(또는 이중 기하급수적으로) 상관관계가 있는 것 같습니다. 재구성 오류의 이러한 차이점이 의미하는 바를 이해하기 위해 네 가지 모델의 재구성 예제를 시각화할 수 있습니다.

In [ ]:
def visualize_reconstructions(model, input_imgs):
    # Reconstruct images
    model.eval()
    with torch.no_grad():
        reconst_imgs = model(input_imgs.to(model.device))
    reconst_imgs = reconst_imgs.cpu()

    # Plotting
    imgs = torch.stack([input_imgs, reconst_imgs], dim=1).flatten(0,1)
    grid = torchvision.utils.make_grid(imgs, nrow=4, normalize=True, range=(-1,1))
    grid = grid.permute(1, 2, 0)
    plt.figure(figsize=(7,4.5))
    plt.title(f"Reconstructed from {model.hparams.latent_dim} latents")
    plt.imshow(grid)
    plt.axis('off')
    plt.show()

In [ ]:
input_imgs = get_train_images(4)
for latent_dim in model_dict:
    visualize_reconstructions(model_dict[latent_dim]["model"], input_imgs)

분명히 가장 작은 잠재 차원은 물체의 거친 모양과 색상에 대한 정보만 저장할 수 있지만 재구성된 이미지는 매우 흐릿하고 재구성에서 원래 물체를 인식하기 어렵습니다. 128개의 특징으로 사진이 흐릿하게 남아 있지만 일부 모양을 다시 인식할 수 있습니다. 256과 384 특징의 차이는 언뜻 보기에는 미미하지만 예를 들어 첫 번째 이미지의 배경을 비교할 때 알 수 있습니다 (384는 256보다 패턴을 더 많이 모델링함).

### Out-of-distribution images

autoencoder의 적용을 계속하기 전에 우리는 autoencoder의 몇 가지 한계를 실제로 살펴볼 수 있습니다. 예를 들어, 데이터 세트의 분포에서 분명히 벗어난 이미지를 재구성하려고 하면 어떻게 될까요? 디코더가 데이터 세트에서 몇 가지 일반적인 패턴을 학습했을 것으로 예상하므로 특히 이러한 패턴을 따르지 않는 이미지를 재구성하지 못할 수 있습니다.

우리가 시도할 수 있는 첫 번째 실험은 노이즈를 재구성하는 것입니다. 따라서 픽셀 값에 대해 균일한 분포에서 무작위로 픽셀이 샘플링된 두 개의 이미지를 만들고 모델의 재구성을 시각화합니다(다른 잠재 차원을 자유롭게 테스트할 수 있음).

In [ ]:
rand_imgs = torch.rand(2, 3, 32, 32) * 2 - 1
visualize_reconstructions(model_dict[256]["model"], rand_imgs)

노이즈의 재구성은 매우 열악하며 몇 가지 거친 패턴을 도입하는 것 같습니다. 입력이 CIFAR 데이터 세트의 패턴을 따르지 않기 때문에 모델은 정확하게 재구성하는 데 문제가 있습니다.

또한 모델이 수동으로 코딩된 다른 패턴을 얼마나 잘 재구성할 수 있는지 확인할 수 있습니다.

In [ ]:
plain_imgs = torch.zeros(4, 3, 32, 32)

# Single color channel
plain_imgs[1,0] = 1
# Checkboard pattern
plain_imgs[2,:,:16,:16] = 1
plain_imgs[2,:,16:,16:] = -1
# Color progression
xx, yy = torch.meshgrid(torch.linspace(-1,1,32), torch.linspace(-1,1,32))
plain_imgs[3,0,:,:] = xx
plain_imgs[3,1,:,:] = yy

visualize_reconstructions(model_dict[256]["model"], plain_imgs)

단일 색상 채널에 눈에 띄는 노이즈가 포함되어 있지만 평범하고 일정한 이미지가 비교적 양호하게 재구성됩니다. 체크보드 패턴의 단단한 경계선은 의도한 것만큼 날카롭지 않으며 색상 진행도 CIFAR의 실제 사진에서는 이러한 패턴이 발생하지 않기 때문입니다.

일반적으로 오토인코더는 손실 함수로 MSE를 선택하기 때문에 고주파 노이즈(즉, 몇 개의 픽셀에서 갑작스러운 큰 변화)를 재구성하는 데 실패하는 경향이 있습니다. 저주파 노이즈의 경우 몇 픽셀의 오정렬이 원본 이미지와 큰 차이를 나타내지 않습니다. 그러나 잠재 차원이 커질수록 이 고주파 노이즈를 더 정확하게 재구성할 수 있습니다.

### Generating new images

Variational 오토인코더는 가우스 분포를 따르도록 잠재 공간을 정규화하기 때문에 오토인코더의 generator 버전입니다. 그러나 바닐라 오토인코더에서는 잠재 벡터에 대한 제한이 없습니다. 무작위로 샘플링된 잠재 벡터를 디코더에 실제로 입력하면 어떻게 될까요? 아래에서 알아봅시다.

In [ ]:
model = model_dict[256]["model"]
latent_vectors = torch.randn(8, model.hparams.latent_dim, device=model.device)
with torch.no_grad():
    imgs = model.decoder(latent_vectors)
    imgs = imgs.cpu()

grid = torchvision.utils.make_grid(imgs, nrow=4, normalize=True, range=(-1,1), pad_value=0.5)
grid = grid.permute(1, 2, 0)
plt.figure(figsize=(8,5))
plt.imshow(grid)
plt.axis('off')
plt.show()

우리가 볼 수 있듯이 생성된 이미지는 사실적인 이미지보다 예술처럼 보입니다.  오토인코더는 재구성에 가장 적합한 방식으로 잠재 공간을 구조화할 수 있으므로 가능한 모든 잠재 벡터를 사실적인 이미지에 매핑할 이유가 없습니다. 게다가, 잠재 공간의 분포는 우리에게 알려지지 않았으며 반드시 정규 분포를 따르는 것은 아닙니다. 따라서 우리는 바닐라 오토인코더가 실제로 생성적이지 않다는 결론을 내릴 수 있습니다.

## Finding visually similar images

오토인코더의 한 가지 응용 프로그램은 이미지 기반 검색 엔진을 구축하여 시각적으로 유사한 이미지를 검색하는 것입니다. 이것은 모든 이미지를 잠재 차원으로 표현하고 이 영역에서 가장 가까운 $K$ 이미지를 찾아 수행할 수 있습니다. 이러한 검색 엔진의 첫 번째 단계는 모든 이미지를 $z$로 인코딩하는 것입니다. 다음에서는 훈련 세트를 검색 코퍼스로 사용하고 테스트 세트를 시스템에 대한 쿼리로 사용합니다.

<span style="color: #880000">(경고: 다음 셀은 약한 CPU 전용 시스템의 경우 계산량이 많을 수 있습니다. 강력한 컴퓨터가 없고 Google Colab에 있지 않은 경우 건너뛰는 것이 좋습니다. 다음 셀을 실행하고 채워진 노트북에 표시된 결과에 의존)</span>

In [ ]:
# We use the following model throughout this section.
# If you want to try a different latent dimensionality, change it here!
model = model_dict[128]["model"]

In [ ]:
def embed_imgs(model, data_loader):
    # Encode all images in the data_laoder using model, and return both images and encodings
    img_list, embed_list = [], []
    model.eval()
    for imgs, _ in tqdm(data_loader, desc="Encoding images", leave=False):
        with torch.no_grad():
            z = model.encoder(imgs.to(model.device))
        img_list.append(imgs)
        embed_list.append(z)
    return (torch.cat(img_list, dim=0), torch.cat(embed_list, dim=0))

train_img_embeds = embed_imgs(model, train_loader)
test_img_embeds = embed_imgs(model, test_loader)


모든 이미지를 인코딩한 후 가장 가까운 $K$ 이미지를 찾아 반환(또는 플롯)하는 함수를 작성하면 됩니다.

In [ ]:
def find_similar_images(query_img, query_z, key_embeds, K=8):
    # Find closest K images. We use the euclidean distance here but other like cosine distance can also be used.
    dist = torch.cdist(query_z[None,:], key_embeds[1], p=2)
    dist = dist.squeeze(dim=0)
    dist, indices = torch.sort(dist)
    # Plot K closest images
    imgs_to_display = torch.cat([query_img[None], key_embeds[0][indices[:K]]], dim=0)
    grid = torchvision.utils.make_grid(imgs_to_display, nrow=K+1, normalize=True, range=(-1,1))
    grid = grid.permute(1, 2, 0)
    plt.figure(figsize=(12,3))
    plt.imshow(grid)
    plt.axis('off')
    plt.show()

In [ ]:
# Plot the closest images for the first N test images as example
for i in range(8):
    find_similar_images(test_img_embeds[0][i], test_img_embeds[1][i], key_embeds=train_img_embeds)

오토인코더를 기반으로 테스트 입력에 대해 유사한 이미지를 많이 검색할 수 있음을 알 수 있습니다. 특히 4행에서 우리는 일부 테스트 이미지가 우리가 생각한 훈련 세트와 크게 다르지 않을 수 있음을 알 수 있습니다(같은 포스터, 단지 다른 스케일링/컬러 스케일링). 또한 모델에 레이블을 지정하지 않았지만 잠재적인 공간(비행기 + 배, 동물 등)의 다른 부분에서 다른 클래스를 클러스터링할 수 있음을 알 수 있습니다. 이것이 오토인코더가 깊은 네트워크에 대한 사전 훈련 전략으로 사용될 수 있는 이유입니다. 특히 레이블이 지정되지 않은 이미지가 많은 경우(종종 그런 경우가 많습니다).

### Tensorboard clustering

잠재 공간에서 이미지의 유사성을 탐색하는 또 다른 방법은 PCA 또는 T-SNE와 같은 차원 축소 방법입니다. 운 좋게도 Tensorboard는 이에 대한 멋진 인터페이스를 제공하며 다음에서 사용할 수 있습니다.

In [ ]:
# We use the following model throughout this section.
# If you want to try a different latent dimensionality, change it here!
model = model_dict[128]["model"]

In [ ]:
# Create a summary writer
writer = SummaryWriter("tensorboard/")

'add_embedding' 기능을 사용하면 클러스터링을 수행할 수 있는 TensorBoard에 고차원 특징 벡터를 추가할 수 있습니다. 함수에서 제공해야 하는 것은 특징 벡터, 레이블과 같은 추가 메타데이터, 클러스터링에서 특정 이미지를 식별할 수 있도록 원본 이미지입니다.

In [ ]:
## In case you obtain the following error in the next cell, execute the import statements and last line in this cell
##   AttributeError: module 'tensorflow._api.v2.io.gfile' has no attribute 'get_filesystem'

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile


# Note: the embedding projector in tensorboard is computationally heavy.
# Reduce the image amount below if your computer struggles with visualizing all 10k points
NUM_IMGS = len(test_set)

writer.add_embedding(test_img_embeds[1][:NUM_IMGS], # Encodings per image
                     metadata=[test_set[i][1] for i in range(NUM_IMGS)], # Adding the labels per image to the plot
                     label_img=(test_img_embeds[0][:NUM_IMGS]+1)/2.0) # Adding the original images to the plot

마지막으로 텐서보드를 실행하여 이미지 간의 유사성을 탐색할 수 있습니다.

In [ ]:
%tensorboard --logdir tensorboard/

다음 이미지와 유사한 것을 볼 수 있어야 합니다. 프로젝터가 비어 있는 경우 Jupyter 노트북 외부에서 TensorBoard를 시작하십시오.

<center><img src="https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial9/tensorboard_projector_screenshot.jpeg?raw=1" width="70%"/></center>

전반적으로, 우리는 모델이 실제로 시각적으로 유사한 이미지를 클러스터링한 것을 볼 수 있습니다. 특히 배경색은 인코딩에서 중요한 요소인 것 같습니다. 이것은 배경이 평균 이미지에서 픽셀의 절반 이상을 담당하기 때문에 선택한 손실 함수(여기서는 픽셀 수준의 평균 제곱 오차)와 관련이 있습니다. 따라서 모델은 배경에 초점을 맞추는 법을 배웁니다. 그럼에도 불구하고 인코딩이 레이블을 보지는 않았지만 잠재 공간에서 두 개의 클래스를 분리한다는 것을 알 수 있습니다. 이것은 자동 인코딩이 분류 전에 "사전 훈련"/전이 학습 작업으로도 사용될 수 있음을 다시 보여줍니다.

In [ ]:
# Closing the summary writer
writer.close()

## Conclusion

이 자습서에서는 작은 RGB 이미지에 자체 오토인코더를 구현하고 모델의 다양한 속성을 탐색했습니다. VAE와 달리 기본 AE는 생성적이지 않으며 MSE 손실 함수에서 작동할 수 있습니다. 이렇게 하면 훈련하기가 더 쉬워집니다. 픽셀 거리를 넘어 시각적으로 유사한 이미지를 찾을 때 보았듯이 두 버전의 AE 모두 차원 축소에 사용할 수 있습니다. 오토인코더는 여전히 잡음 제거 및 압축과 같은 많은 응용 프로그램에서 사용되므로, AE는 모든 딥 러닝 엔지니어/연구원이 숙지해야 하는 필수 도구입니다.